In [1]:

%matplotlib widget

import cell_model_base as cs_model
import detect_peaks as dp
import numpy as np
import matplotlib.pyplot as plt
import place_cell_base as pl

mean_video_srate = 100.
total_Time = 500
_lambda = 10

x_coordinates,y_coordinates,speed,timevector = cs_model.generate_randomWalk2(input_srate = mean_video_srate,input_total_Time = total_Time,heading_srate = 10., speed_srate = 5., rho1  = 1,sigma = 0.02,mu_e  = 0.,smooth_coeff = 0.5)
All_arrival_time,I_timestamps = cs_model.generate_arrivals(_lambda,total_Time)

modulated_timestamps = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)
# modulated_timestamps1 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 50,y_center = 50, s = 5)
# modulated_timestamps2 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 10, s = 5)
# modulated_timestamps3 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 10,y_center = 90, s = 5)
# modulated_timestamps4 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 90, s = 5)
# modulated_timestamps5 = cs_model.digitize_spiketimes(x_coordinates,y_coordinates,I_timestamps,x_center = 90,y_center = 10, s = 5)

# modulated_timestamps = np.unique(np.concatenate([modulated_timestamps1,modulated_timestamps2,modulated_timestamps3,modulated_timestamps4,modulated_timestamps5]))
modulated_timestamps = np.random.choice(modulated_timestamps,50,replace=False)

calcium_imag,timevector = cs_model.generate_CalciumSignal(modulated_timestamps,noise_level = 0.01, b = 5.)
calcium_events = modulated_timestamps.shape[0]
I_peaks = dp.detect_peaks(calcium_imag,mpd=0.5*mean_video_srate,mph=1.*np.nanstd(calcium_imag))

plt.figure()
plt.plot(x_coordinates,y_coordinates,'k.')
plt.plot(x_coordinates[I_peaks],y_coordinates[I_peaks],'r.')
plt.show()

place_cell = pl.PlaceCell(RatSession='Sst',day = 1, ch = 1, dataset = 'Exploration',mean_video_srate = 100.,mintimespent =  0,
                          minvisits = 0, speed_threshold = 10, nbins_cal = 10, num_surrogates = 200,num_cores = 20, nbins_pos_x = 10, 
                          nbins_pos_y = 10, placefield_nbins_pos_x = 50, placefield_nbins_pos_y = 50)

PlaceCellObject = place_cell.run_placeMetrics(calcium_imag,timevector,x_coordinates,y_coordinates)


plt.figure()
plt.pcolormesh(PlaceCellObject['x_grid'],PlaceCellObject['y_grid'],PlaceCellObject['signalMap'])
plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/apolo/Documents/DataAnalysis/Projects/CalciumSignalAnalysis/place_cell_base.py:152: RuntimeWarning: Mean of empty slice
  calcium_mean_occupancy[yy,xx] = np.nanmean(mean_calcium_to_behavior_speed[np.logical_and(check_x_ocuppancy,check_y_ocuppancy)])


File not saved!


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:

place_cell = pl.PlaceCell(RatSession='Sst',day = 1, ch = 1, dataset = 'Exploration',mean_video_srate = 100.,mintimespent =  0,
                          minvisits = 5, speed_threshold = 10, nbins_cal = 10, num_surrogates = 200,num_cores = 20, nbins_pos_x = 10, 
                          nbins_pos_y = 10, placefield_nbins_pos_x = 50, placefield_nbins_pos_y = 50)

PlaceCellObject = place_cell.run_placeMetrics(calcium_imag,timevector,x_coordinates,y_coordinates)


plt.figure()
plt.pcolormesh(PlaceCellObject['x_grid'],PlaceCellObject['y_grid'],PlaceCellObject['signalMap'])
plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()


/home/apolo/Documents/DataAnalysis/Projects/CalciumSignalAnalysis/place_cell_base.py:152: RuntimeWarning: Mean of empty slice
  calcium_mean_occupancy[yy,xx] = np.nanmean(mean_calcium_to_behavior_speed[np.logical_and(check_x_ocuppancy,check_y_ocuppancy)])


File not saved!


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
mean_video_srate = 100.
mintimespent =  0
minvisits = 2
speed_threshold = 1
nbins_cal = 10
num_surrogates = 200
num_cores = 20
nbins_pos_x = 10
nbins_pos_y = 10
placefield_nbins_pos_x = 50
placefield_nbins_pos_y = 50
        
        
mean_calcium_to_behavior = calcium_imag
track_timevector = timevector


x_coordinates_valid, y_coordinates_valid, mean_calcium_to_behavior_valid, track_timevector_valid = place_cell.get_valid_timepoints(mean_calcium_to_behavior,x_coordinates,y_coordinates,track_timevector,speed_threshold)

calcium_signal_binned_signal = place_cell.get_binned_signal(mean_calcium_to_behavior_valid,nbins_cal)

x_grid,y_grid,x_center_bins,y_center_bins = place_cell.get_position_grid(x_coordinates,y_coordinates,nbins_pos_x,nbins_pos_y)

position_binned = place_cell.get_binned_2Dposition(x_coordinates_valid,y_coordinates_valid,nbins_pos_x,nbins_pos_y)

nbins_pos = nbins_pos_x*nbins_pos_y

mutualInfo_original = place_cell.mutualInformation(position_binned,calcium_signal_binned_signal,nbins_pos,nbins_cal)

mutualInfo_permutation = place_cell.get_perm_distribution(position_binned,calcium_signal_binned_signal,nbins_pos,nbins_cal,num_cores,num_surrogates)

mutualInfo_zscored = place_cell.get_mutualInfo_zscore(mutualInfo_original,mutualInfo_permutation)



In [45]:
placefield_nbins_pos_x

50

In [4]:
x_grid_pc,y_grid_pc,x_center_bins_pc,y_center_bins_pc = place_cell.get_position_grid(x_coordinates,y_coordinates,placefield_nbins_pos_x,placefield_nbins_pos_y)

visits_occupancy = place_cell.get_visits(x_coordinates,y_coordinates,x_grid_pc,y_grid_pc,x_center_bins_pc,y_center_bins_pc)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 2., 1.],
       [1., 3., 4., ..., 2., 4., 1.],
       ...,
       [3., 3., 4., ..., 3., 3., 1.],
       [1., 2., 3., ..., 1., 2., 0.],
       [2., 3., 3., ..., 3., 3., 0.]])

In [6]:
x_coordinates_valid, y_coordinates_valid, mean_calcium_to_behavior_valid, track_timevector_valid = place_cell.get_valid_timepoints(mean_calcium_to_behavior,x_coordinates,y_coordinates,track_timevector,speed_threshold)


calcium_mean_occupancy,calcium_mean_occupancy_smoothed,position_occupancy,visits_occupancy,x_grid_pc,y_grid_pc,x_center_bins_pc,y_center_bins_pc = place_cell.placeField(mean_calcium_to_behavior_valid,x_coordinates_valid,y_coordinates_valid,track_timevector_valid,mean_video_srate,mintimespent,minvisits,placefield_nbins_pos_x,placefield_nbins_pos_y)


/home/apolo/Documents/DataAnalysis/Projects/CalciumSignalAnalysis/place_cell_base.py:152: RuntimeWarning: Mean of empty slice
  calcium_mean_occupancy[yy,xx] = np.nanmean(mean_calcium_to_behavior_speed[np.logical_and(check_x_ocuppancy,check_y_ocuppancy)])


In [9]:



plt.figure()
plt.pcolormesh(PlaceCellObject['x_grid'],PlaceCellObject['y_grid'],visits_occupancy>10)
plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:

plt.figure()
plt.pcolormesh(PlaceCellObject['x_grid'],PlaceCellObject['y_grid'],PlaceCellObject['signalMap'])
plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:

plt.figure()
plt.pcolormesh(PlaceCellObject['x_grid'],PlaceCellObject['y_grid'],PlaceCellObject['signalMapSmoothed'])
plt.plot(x_coordinates[PlaceCellObject['events_index']],y_coordinates[PlaceCellObject['events_index']],'r.')
plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:

center_x = PlaceCellObject['x_grid'][0:-1] + np.diff(PlaceCellObject['x_grid'])
center_y = PlaceCellObject['y_grid'][0:-1] + np.diff(PlaceCellObject['y_grid'])

# (X,Y) = np.meshgrid(center_x,center_y)

plt.figure()
cnt = plt.contourf(center_x,center_y,PlaceCellObject['signalMapSmoothed'],100)
for c in cnt.collections:
    c.set_edgecolor("face")


plt.colorbar()
plt.title('MI = ' + str(np.round(PlaceCellObject['mutualInfo_zscored'],2)) + '; Events = ' + str(calcium_events))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …